In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix)

C:\Users\linyu\AppData\Local\Temp\ipykernel_31556\3459847097.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Read data
df = pd.read_csv('dataset_phishing.csv')

In [3]:
features = ['shortest_word_path',
 'ratio_intMedia',
 'links_in_tags',
 'nb_hyphens',
 'page_rank',
 'avg_word_path',
 'ratio_extHyperlinks',
 'longest_words_raw',
 'google_index',
 'length_hostname',
 'longest_word_host',
 'domain_registration_length',
 'nb_www',
 'nb_underscore',
 'nb_dots',
 'ratio_extMedia',
 'phish_hints',
 'domain_in_title',
 'web_traffic',
 'safe_anchor',
 'nb_space',
 'shortening_service',
 'ip',
 'domain_age',
 'nb_qm',
 'nb_hyperlinks',
 'nb_slash']

In [4]:
X = df[features]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

le = LabelEncoder()
y = le.fit_transform(df["status"])

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

# Define the parameters that you want to easily modify
params = {
    'hidden_layer_sizes': (100, 50),   # Structure of hidden layers
    'activation': 'relu',              # Activation function: 'relu', 'tanh', 'logistic'
    'solver': 'adam',                  # Optimizer: 'adam', 'sgd', 'lbfgs'
    'alpha': 0.006,                     # L2 regularization term
    'batch_size': 'auto',              # Batch size for optimization
    'learning_rate': 'adaptive',       # Learning rate schedule: 'constant', 'invscaling', 'adaptive'
    'learning_rate_init': 0.008,        # Initial learning rate
    'max_iter': 10000,                   # Max number of iterations
    'tol': 1e-4,                       # Tolerance for optimization stopping criteria
    'random_state': 42,                # Random seed for reproducibility
    'verbose': True                    # Whether to print progress during training
}

    
# Initialize the MLPClassifier with the parameters from the dictionary
mlp = MLPClassifier(**params)

# Train the model
mlp.fit(X_train, y_train)

# Make predictions on the test set
y_pred = mlp.predict(X_test)

# Evaluate the model
recall = recall_score(y_test, y_pred, average='binary')

print(recall)

Iteration 1, loss = 0.20483984
Iteration 2, loss = 0.13824240
Iteration 3, loss = 0.13291813
Iteration 4, loss = 0.11482660
Iteration 5, loss = 0.10888418
Iteration 6, loss = 0.10234306
Iteration 7, loss = 0.11781868
Iteration 8, loss = 0.09727804
Iteration 9, loss = 0.13287715
Iteration 10, loss = 0.10378632
Iteration 11, loss = 0.10520189
Iteration 12, loss = 0.09330740
Iteration 13, loss = 0.08347263
Iteration 14, loss = 0.08157335
Iteration 15, loss = 0.07906299
Iteration 16, loss = 0.08607999
Iteration 17, loss = 0.07745353
Iteration 18, loss = 0.07048087
Iteration 19, loss = 0.06788784
Iteration 20, loss = 0.06781601
Iteration 21, loss = 0.06194830
Iteration 22, loss = 0.05851555
Iteration 23, loss = 0.05766441
Iteration 24, loss = 0.06253596
Iteration 25, loss = 0.05440714
Iteration 26, loss = 0.06145243
Iteration 27, loss = 0.05525083
Iteration 28, loss = 0.05565973
Iteration 29, loss = 0.05335214
Iteration 30, loss = 0.04575087
Iteration 31, loss = 0.04540362
Iteration 32, los